In [2]:
# import libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime


In [3]:
today = datetime.today().strftime('%Y-%m-%d')

url = "https://www.eloratings.net/fixtures"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
driver.get(url)
driver.implicitly_wait(10)

rows = driver.find_elements(By.CSS_SELECTOR, 'div.ui-widget-content.slick-row')
data = []

# Loop through each row and extract the data
for row in rows:
    cells = row.find_elements(By.CSS_SELECTOR, 'div.slick-cell')

    date = cells[0].get_attribute('innerHTML').replace('<br>', ' ').strip()

    teams_html = cells[1].get_attribute('innerHTML')
    teams_soup = BeautifulSoup(teams_html, 'html.parser')
    teams = teams_soup.find_all('a')
    teams_list = [team.get_text() for team in teams]

    location_html = cells[2].get_attribute('innerHTML')
    location_soup = BeautifulSoup(location_html, 'html.parser')
    location = location_soup.get_text(separator=' ').strip()  # Use get_text to extract all text content


    rank = cells[3].get_attribute('innerHTML').replace('<br>', ' ').strip()
    elo = cells[4].get_attribute('innerHTML').replace('<br>', ' ').strip()
    win_pct = cells[5].get_attribute('innerHTML').replace('<br>', ' ').strip()


    match_data = {
        'date': date,
        'team1': teams_list[0],
        'team2': teams_list[1],
        'location': location,
        'rank': rank,
        'elo': elo,
        'win_pct': win_pct,
         "scrape_date":today
    }

# Append the dictionary to the list
    data.append(match_data)

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
today = today.replace("-", "_")
df.to_csv(f'../Data/upcoming_matches_{today}.csv', index=False) #save historical data
df.to_csv(f'../Data/upcoming_matches.csv', index=False) #save current version

# Close the WebDriver
driver.quit()